In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr, kendalltau

import warnings
warnings.filterwarnings("ignore")

# Set global aesthetics
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10,6)


In [ ]:

# Load Excel file
file_path = 'climate_change_dataset.xlsx'

# Check sheet names
xls = pd.ExcelFile(file_path)
print(xls.sheet_names)

# Load sheets (update names as per actual file)
df_disasters = pd.read_excel(xls, sheet_name="EADRF")
df_temp = pd.read_excel(xls, sheet_name="CCATA")


In [ ]:

# Disaster Dataset
df_disasters.dropna(how='all', axis=1, inplace=True)
df_disasters['Year'] = pd.to_numeric(df_disasters['Year'], errors='coerce')
df_disasters = df_disasters.dropna(subset=['Year'])

# Temperature Dataset
df_temp['Year'] = pd.to_numeric(df_temp['Year'], errors='coerce')
df_temp = df_temp.dropna(subset=['Year'])

# Merge on Year
df_merged = pd.merge(df_disasters, df_temp, on='Year', how='inner')


In [ ]:

# Disaster Occurrences Over Time
df_occurrence = df_disasters.groupby('Year')['Disaster Type'].count().reset_index()
plt.plot(df_occurrence['Year'], df_occurrence['Disaster Type'], marker='o')
plt.title("Natural Disaster Occurrences Over Time")
plt.xlabel("Year")
plt.ylabel("Number of Disasters")
plt.show()


In [ ]:

# Global Temperature Anomaly Over Time
plt.plot(df_temp['Year'], df_temp['Temperature Anomaly'], color='red')
plt.title("Global Temperature Anomaly Over Time")
plt.xlabel("Year")
plt.ylabel("Temperature Anomaly (°C)")
plt.show()


In [ ]:

# Economic Damage by Disaster Type
damage_df = df_disasters.groupby('Disaster Type')['Economic Damage (USD)'].sum().sort_values(ascending=False)
damage_df.plot(kind='bar', color='orange')
plt.title("Total Economic Damage by Disaster Type")
plt.ylabel("Economic Damage in USD")
plt.xticks(rotation=45)
plt.show()


In [ ]:

# Correlation Heatmap
corr_cols = ['Disaster Count', 'Deaths', 'Injured', 'Homeless', 'Economic Damage (USD)', 'Temperature Anomaly']
df_corr = df_merged[corr_cols].dropna()
sns.heatmap(df_corr.corr(method='pearson'), annot=True, cmap='coolwarm')
plt.title("Pearson Correlation Heatmap")
plt.show()


In [ ]:

# Correlation between Disaster Count and Temperature Anomaly
pearson = pearsonr(df_merged['Disaster Count'], df_merged['Temperature Anomaly'])
spearman = spearmanr(df_merged['Disaster Count'], df_merged['Temperature Anomaly'])
kendall = kendalltau(df_merged['Disaster Count'], df_merged['Temperature Anomaly'])

print("📈 Correlation Between Disaster Count and Temperature Anomaly")
print(f"Pearson: {pearson[0]:.4f}, p-value: {pearson[1]:.4e}")
print(f"Spearman: {spearman[0]:.4f}, p-value: {spearman[1]:.4e}")
print(f"Kendall: {kendall[0]:.4f}, p-value: {kendall[1]:.4e}")


In [ ]:

top_countries = df_disasters['Country'].value_counts().head(10)
top_countries.plot(kind='bar', title="Top 10 Countries by Disaster Count", color='teal')
plt.ylabel("Number of Disasters")
plt.xticks(rotation=45)
plt.show()


In [ ]:

df_merged.to_csv("merged_climate_disasters.csv", index=False)
